### 코랩에서 한글 사용하기
- 한글 글꼴 설치 => [런타임]-[런타임 다시 시작] 클릭


### 서울시 유가정보(11개 구) 분서

#### [미션1] 서울시 유가정보 가져와 1개로 합치기
1. "/content" 폴더에 "data" 폴더 생성후 파일 업로드 
2. data 폴더에 있는 데이터 읽어와 oildf에 모두 저장
3. 데이터는 concat()을 이용해 데이터의 아래에 추가
4. encoding: cp949 또는 utf8이 같이 있어 에러 발생
        try: ~ except: ~ 로 해결



- data 폴더에 있는 파일명/하위폴더명 가져오기: os모듈
- for문을 이용 첫번째 파일부터 순차적으로 읽어와 oildf에 concat 진행
- 파일을 읽을시 encoding 에러
        try: 
            utf-8 형식 읽어오기
        except:
            cp949 형식 읽어오기
- 빈 DataFrame 변수 생성
        oildf=pd.DataFrame()

In [13]:
import pandas as pd
import os

fnames=os.listdir("./data")

oildf = pd.DataFrame()

for fname in fnames:
    try:
        df0=pd.read_csv("./data/" + fname, header=2)
    except:
        df0=pd.read_csv("./data/" + fname, header=2, encoding="cp949")
        
    oildf=pd.concat([oildf, df0])  # 옵션: ignore_index=True : index 재설정

del df0

oildf.reset_index(drop=True, inplace=True)

display(oildf.head(3))
oildf.info()

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,현대오일뱅크(주)직영 산성셀프주유소,서울 강남구 헌릉로 730,현대오일뱅크,222264963,Y,2140.0,1955.0,1900.0,NaN
1,서울특별시,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,24456870,Y,NaN,1955.0,1900.0,NaN
2,서울특별시,방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,24593434,Y,NaN,1987.0,1919.0,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      225 non-null    object
 1   상호      225 non-null    object
 2   주소      225 non-null    object
 3   상표      225 non-null    object
 4   전화번호    225 non-null    object
 5   셀프여부    225 non-null    object
 6   고급휘발유   203 non-null    object
 7   휘발유     220 non-null    object
 8   경유      221 non-null    object
 9   실내등유    189 non-null    object
dtypes: object(10)
memory usage: 17.7+ KB


In [14]:
print(fnames)

['지역_위치별(주유소) (1).csv', '지역_위치별(주유소) (6).csv', '지역_위치별(주유소) (10).csv', '지역_위치별(주유소) (7).csv', '지역_위치별(주유소).csv', '지역_위치별(주유소) (4).csv', '지역_위치별(주유소) (8).csv', '지역_위치별(주유소) (9).csv', '지역_위치별(주유소) (5).csv', '지역_위치별(주유소) (2).csv', '지역_위치별(주유소) (3).csv']


#### [미션2] 데이터 전처리
- NaN 값 포함된 데이터 전체 삭제
- "구" 필드 생성하기
- 데이터 출력 필드: 상호, 상표, 구, 셀프여부, 휘발유, 경우 만 출력
- "휘발유", "경유": float64로 변경
- "휘발유", "경유" 데이터의 "-"를 평균값으로 수정


In [15]:
## 필드명 NaN 갯수 확인
oildf.isna().sum()

지역        0
상호        0
주소        0
상표        0
전화번호      0
셀프여부      0
고급휘발유    22
휘발유       5
경유        4
실내등유     36
dtype: int64

In [16]:
## loc[] 함수를 이용해 주소에서 구 추출하기
gutxt = []
for i in oildf.index:
    gutxt.append( oildf.loc[i, '주소'].split(" ")[1] )

In [17]:
# 한줄쓰기로 표현
gutxt=[ oildf.loc[i, '주소'].split(" ")[1] for i in oildf.index ]

In [18]:
# iloc[] 함수를 이용해 주소에서 구 추출하기
gutxt=[ oildf.iloc[i, 2].split(" ")[1] for i in range(len(oildf)) ]

In [19]:
# "구" 필드 추가
oildf["구"]=gutxt

In [20]:
# 필드명으로 DataFrame 생성하기
oildf1=oildf[['상호','상표',"구",'셀프여부', '휘발유', '경유']]

In [21]:
# iloc를 이용한 DataFrame 생성하기
oildf2=oildf.iloc[:, [1, 3,-1, 5, 7, 8]]
oildf2.dtypes

상호      object
상표      object
구       object
셀프여부    object
휘발유     object
경유      object
dtype: object

In [22]:
display(oildf1.head(1))
display(oildf2.head(1))

,상호,상표,구,셀프여부,휘발유,경유
0,현대오일뱅크(주)직영 산성셀프주유소,현대오일뱅크,강남구,Y,1955.0,1900.0


,상호,상표,구,셀프여부,휘발유,경유
0,현대오일뱅크(주)직영 산성셀프주유소,현대오일뱅크,강남구,Y,1955.0,1900.0


In [23]:
# NaN값 0으로 변경
oildf2=oildf2.fillna(0)

In [24]:
# 휘발유/경우 데이터에서 "-" 값만 찾아 0으로 변경
oildf2.loc[oildf2.휘발유=="-",'휘발유']=0
oildf2.loc[oildf2["경유"]=="-",'경유']=0

Index(['지역', '상호', '주소', '상표', '전화번호', '셀프여부', '고급휘발유', '휘발유', '경유', '실내등유'],

In [25]:
# 고유값 확인하기
oildf2['휘발유'].unique()

array([1955.0, 1987.0, 2028.0, 2038.0, 2040.0, 2045.0, 2047.0, 2052.0,
       2055.0, 2062.0, 2079.0, 2089.0, 2097.0, 2108.0, 2158.0, 2194.0,
       2228.0, 2239.0, 2243.0, 2288.0, 2296.0, 2329.0, 2380.0, 2393.0,
       2394.0, 2398.0, 2545.0, 0, 1978.0, 1985.0, 1988.0, 1993.0, 1997.0,
       1998.0, 1999.0, 2023.0, 2025.0, 2027.0, 2148.0, '1949', '1954',
       '1973', '1985', '2014', '2018', '2025', '2027', '2048', '2068',
       '1945', '1995', '1998', '1999', '2007', '2011', '2028', '2039',
       '2045', '2053', '2065', '2069', '2085', 2039, 2059, 2069, 2199,
       2299, 1989, 1995, 2009, '2089', '2119', '2120', '2345', '2378',
       '2395', '2398', 1926, 1975, 1979, 1986, 1996, 2018, 2029, 2048,
       2099, 2177, 2008, 2010, 2020, 2022, 2031, 2035, 2071, 2073, 2074,
       2088, 2095, 2138, 2198, 2238, 2298, 2477, 2065.0, 2098.0, 2172.0,
       1968, 2015, 2090], dtype=object)

In [26]:
# 휘발유/경우 실수형 자료로 변경하기
oildf2=oildf2.astype({"휘발유":float, "경유":float})
oildf2.dtypes  # 데이터 타입 확인하기

상호       object
상표       object
구        object
셀프여부     object
휘발유     float64
경유      float64
dtype: object

In [27]:
# 휘발유/경유 데이터에서 0인 값을 평균값으로 변경하기
oildf2.loc[oildf2.휘발유 == 0,'휘발유'] = oildf2.loc[oildf2.휘발유 != 0,'휘발유'].mean()
oildf2.loc[oildf2["경유"] == 0,'경유'] = oildf2.loc[oildf2["경유"] != 0,'경유'].mean()

In [28]:
# 0인 값이 인는지 확인
oildf2.loc[oildf2["경유"] == "-", :]   

,상호,상표,구,셀프여부,휘발유,경유


In [29]:
# oildf2에서 휘발유 최고가 검색
oildf2[oildf2["휘발유"] == oildf2["휘발유"].max()]

,상호,상표,구,셀프여부,휘발유,경유
33,제이제이주유소,현대오일뱅크,강남구,N,2545.0,2335.0


In [30]:
# "구" 필드를 기준으로 평균 값 계산
guGroup=oildf2.groupby(by="구").mean()   # 그룹 항목이 index로 설정
guGroup.head()

,휘발유,경유
구,,
강남구,2147.099888,2038.561369
강동구,2075.842660,1960.161206
강북구,2001.230769,1912.153846
노원구,2035.933333,1938.666667
도봉구,2020.994368,1934.792049


In [31]:
# 강북구 데이터만 출력
guGroup.loc[["강북구"]]

,휘발유,경유
구,,
강북구,2001.230769,1912.153846


In [32]:
# 강북구 / 양천구 출력
guGroup.loc[["강북구","양천구"]]

,휘발유,경유
구,,
강북구,2001.230769,1912.153846
양천구,2001.760000,1929.840000


In [33]:
guGroup.index

Index(['강남구', '강동구', '강북구', '노원구', '도봉구', '마포구', '서초구', '송파구', '양천구', '은평구',
       '종로구'],
      dtype='object', name='구')

In [34]:
# 강북구 ~ 서초구까지 의 휘발유값 출력
guGroup.loc["강북구":"서초구", "휘발유"]

구
강북구    2001.230769
노원구    2035.933333
도봉구    2020.994368
마포구    2071.750000
서초구    2068.696970
Name: 휘발유, dtype: float64

In [35]:
# 구별 주유소 갯수
oildf2.groupby(["구"])["상표"].count()

구
강남구    37
강동구    14
강북구    13
노원구    15
도봉구    18
마포구    12
서초구    33
송파구    32
양천구    25
은평구    17
종로구     9
Name: 상표, dtype: int64

In [36]:
# 구별/상표별 주유소 갯수
oildf2.groupby(["구", "상표"])[["휘발유", "경유"]].count()

휘발유  경유
구   상표             
강남구 GS칼텍스     9   9
    SK에너지    15  15
    SOIL      5   5
    현대오일뱅크    8   8
강동구 GS칼텍스     5   5
    S-OIL     2   2
    SK에너지     3   3
    현대오일뱅크    4   4
강북구 GS칼텍스     3   3
    S-OIL     3   3
    SK에너지     3   3
    현대오일뱅크    4   4
노원구 GS칼텍스     6   6
    S-OIL     3   3
    SK에너지     3   3
    현대오일뱅크    3   3
도봉구 GS칼텍스     3   3
    S-OIL     5   5
    SK에너지     3   3
    현대오일뱅크    7   7
마포구 GS칼텍스     1   1
    S-OIL     3   3
    SK에너지     5   5
    현대오일뱅크    3   3
서초구 GS칼텍스    13  13
    S-OIL     5   5
    SK에너지     9   9
    알뜰(ex)    1   1
    현대오일뱅크    5   5
송파구 GS칼텍스     9   9
    S-OIL     5   5
    SK에너지    10  10
    현대오일뱅크    8   8
양천구 GS칼텍스     5   5
    S-OIL     1   1
    SK에너지     9   9
    알뜰주유소     1   1
    현대오일뱅크    9   9
은평구 GS칼텍스     6   6
    S-OIL     2   2
    SK에너지     5   5
    자가상표      1   1
    현대오일뱅크    3   3
종로구 GS칼텍스     2   2
    S-OIL     2   2
    SK에너지     3   3
    현대오일뱅크    2   2

In [37]:
# 구별/상표별 주유소 갯수 계산후 by 값을 데이터로 변경(reset_index() 실행)
oildf2.groupby(["구", "상표"], as_index=False)["휘발유"].count()

,구,상표,휘발유
0,강남구,GS칼텍스,9
1,강남구,SK에너지,15
2,강남구,SOIL,5
3,강남구,현대오일뱅크,8
4,강동구,GS칼텍스,5
5,강동구,S-OIL,2
6,강동구,SK에너지,3
7,강동구,현대오일뱅크,4
8,강북구,GS칼텍스,3
9,강북구,S-OIL,3


In [38]:
# 그룹 안에서 원한는 데이터 확인
oildf2.groupby("구").get_group("노원구")

,상호,상표,구,셀프여부,휘발유,경유
210,한신주유소,SK에너지,노원구,Y,1968.0,1838.0
211,삼육주유소,SK에너지,노원구,Y,1975.0,1845.0
212,태릉솔밭주유소,S-OIL,노원구,Y,1995.0,1895.0
213,화랑대주유소,현대오일뱅크,노원구,Y,1995.0,1895.0
214,이엠석유(주)배꽃나라주유소,GS칼텍스,노원구,Y,2009.0,1909.0
215,현대오일뱅크㈜직영 하계주유소,현대오일뱅크,노원구,N,2015.0,1925.0
216,월계주유소,GS칼텍스,노원구,Y,2015.0,1915.0
217,SK에너지㈜직영 상계주유소,SK에너지,노원구,N,2025.0,1965.0
218,남선석유(주)구도일주유소 불암,S-OIL,노원구,Y,2025.0,1965.0
219,노원주유소,GS칼텍스,노원구,N,2038.0,1949.0


In [41]:
# oildf2 csv 파일로 저장하기
oildf2.to_csv("서울시유류정보.csv", index=False, encoding="cp949")